# Governance Data Preprocessing

In [1]:
### package
import os 
import pandas as pd
import numpy as np
import copy
os.chdir("C:/Users/user/Desktop/Governance/2021")

### 데이터 로딩

In [2]:
### ESG
raw_esg = pd.read_excel("ESG_DATA.xlsx",converters={'종목코드':str})

### DART
dart_name = ['지가경공시','총회ESG','임원','사외이사','회계감사','지배구조','배당공지']
for i in range(1,len(dart_name)+1) :
    globals()['raw_dart{}'.format(i)] = pd.read_excel("DART"+str(dart_name[i-1])+"_2021.xlsx")
raw_dart3 = pd.read_excel("DART임원_2021.xlsx",converters={'종목코드':str})

### KIND
kind_name = ['공시우수법인F','횡령3y','지배구조']
for i in range(1,len(kind_name)+1) :
    globals()['raw_kind{}'.format(i)] = pd.read_excel("KIND"+str(kind_name[i-1])+"_2021.xlsx",converters={'종목코드':str})

### KRX
raw_krx = pd.read_excel("KRX불성실공시3y_2021.xlsx",converters={'종목코드':str})

### KSA
raw_ksa = pd.read_excel("KSA지가경_2021.xlsx")

### 금융지배구조
raw_fgover = pd.read_excel("금융지배구조_2021.xlsx")

### 지속가능경영보고서 지배구조부문
raw_gover = pd.read_excel("지배구조_2021.xlsx")

### 공정위위반
raw_ill = pd.read_excel("공정위위반사항_2021.xlsx")

### 분석대상 key id

In [3]:
### 분석대상 key 변수 : 분석대상에서 SBS미디어홀딩스, 에스케이머티리얼즈 제외
raw_esg = raw_esg[-raw_esg['종목코드'].isin(['101060','036490'])]
code = list(raw_esg['종목코드'])
corp = list(raw_esg['회사명'])
idx_esg = raw_esg[['종목코드','회사명']]
sector_esg = raw_esg[['종목코드','회사명','SICS_Sector_Kr']]

### Governance 데이터 분석대상 Filtering

In [4]:
### 분석대상에 필요한 회사 추출 : 모든 데이터에 회사명에 따른 종목코드 추가

### DART
idx_dart = raw_dart3[['회사명','종목코드']].drop_duplicates(['회사명','종목코드'], keep = 'first')
dart_name = ['지가경공시','총회ESG','임원','사외이사','회계감사','지배구조','배당공지']
for i in [1,2,4,5,6,7] :
    globals()['dart{}'.format(i)] = pd.merge(left = globals()['raw_dart{}'.format(i)], right = idx_dart, how = "left", on = "회사명")
    globals()['dart{}'.format(i)] = globals()['dart{}'.format(i)][globals()['dart{}'.format(i)]['종목코드'].isin(code)]
dart3 = raw_dart3[raw_dart3['종목코드'].isin(code)]

### KIND
kind_name = ['공시우수법인F','횡령3y','지배구조']
for i in range(1,len(kind_name)+1) :
    globals()['kind{}'.format(i)] = globals()['raw_kind{}'.format(i)][globals()['raw_kind{}'.format(i)]['종목코드'].isin(code)]

### KRX
krx = raw_krx[raw_krx['종목코드'].isin(code)]

### KSA
ksa = pd.merge(left = raw_ksa, right = idx_esg, on = '회사명')

### 금융회사지배구조
fgover = pd.merge(left = raw_fgover, right = idx_esg, how = 'left', on = '회사명')

### 지배구조 지가경 데이터
gover = pd.merge(left = raw_gover, right = sector_esg, how = 'left', on = '회사명')

### 공정위 위반
ill = pd.merge(left = raw_ill, right = idx_esg, how = 'left', on = '회사명') 

### Data Preprocessing

In [5]:
### DART

## 지속가능경영보고서 공시여부 : 공시했으면 1, 아니면 0
dart1['지가경공시'] = 1
r_dart1 = dart1.drop_duplicates("종목코드", keep='first')
new_dart1 = r_dart1.drop(['회사명'], axis=1)

## 주주총회 ESG안건 여부 : ESG안건 있으면 1, 없으면 0 (정정여부는 제외)
dart2['주총ESG안건'] = 1
new_dart2 = dart2[['종목코드','주총ESG안건']].drop_duplicates('종목코드',keep='first')

## 임원

# 등기임원 추출
dart3 = dart3[dart3['등기임원여부'] != '미등기임원'][dart3['등기임원여부'] != '미등기']

# 전체 임원 수
total_chief = pd.DataFrame(dart3.groupby('종목코드').size(), columns = ['총이사']) 
total_chief = total_chief.reset_index()

# 여성 임원 수
woman_chief = dart3[dart3['성별'] == '여'].groupby('종목코드').size().reset_index().rename(columns = {0:'여성이사'})

# ESG관련 이사 수 : ESG, 지속가능, 미래전략, 지속경영 키워드가 관련된 ESG이사 추출
chief_esg = list(dart3['담당업무'])
for i in range(dart3.shape[0]) :
    if any(word in chief_esg[i] for word in ['ESG','esg','지속가능','미래전략','지속경영']) :
        chief_esg[i] = 1
    else : chief_esg[i] = 0
        
c = pd.DataFrame({'종목코드':list(dart3['종목코드']),
                  'ESG이사':chief_esg})
esg_chief = c.groupby('종목코드')['ESG이사'].sum().reset_index()

# 대표이사 이사회의장 일치여부 '이사회 독립성' 
# :지속가능경영보고서 데이터와 지배구조 데이터를 결합
dart6_1 = dart6[dart6['Q7'] == 1]
dart6_1['이사회독립성'] = '독립'
new_dart6_1 = pd.concat([gover[['종목코드','이사회독립성']],fgover[['종목코드','이사회독립성']],dart6_1[['종목코드','이사회독립성']]]).drop_duplicates('종목코드', keep = 'first')
# :직위와 담당업무에 이사회의장만, 대표이사만 있는 경우는 독립 그 외는 겸임
ceo1 = dart3[dart3['직위'].str.contains('의장')][-dart3['직위'].str.contains('대표')][-dart3['담당업무'].str.contains('대표')]
ceo2 = dart3[dart3['담당업무'].str.contains('의장')][-dart3['담당업무'].str.contains('대표')][-dart3['직위'].str.contains('대표')]
ceo_chief = pd.concat([ceo1,ceo2]).drop_duplicates(['종목코드'],keep='first')
ceo_chief['이사회독립성'] = "독립"
# :중복 데이터 제거
ceo_chief = ceo_chief[~ceo_chief['회사명'].isin(['현대미포조선','제넥신','대우조선해양'])][['종목코드','이사회독립성']]
# :최종 데이터 = 지속가능경영보고서 & 지배구조데이터 & DART임원 데이터 결합
ceo_chief = pd.concat([new_dart6_1,ceo_chief]).drop_duplicates('종목코드', keep = 'first')

# 사외임원 수
dart4 = dart4.replace("-",0)
r_dart4 = pd.merge(left = total_chief, right = dart4[['종목코드','사외이사 수']], how = 'left', on = '종목코드') 
r_dart4['사외이사비율'] = r_dart4['사외이사 수'].astype(float)/r_dart4['총이사'].astype(float)
new_dart4 = r_dart4[['종목코드','사외이사비율']]

## total : 최종데이터
dart3_1 = pd.merge(left = total_chief, right = ceo_chief, on='종목코드', how='left').fillna("겸임")
dart3_2 = pd.merge(left = dart3_1, right = esg_chief, on='종목코드', how='left')
dart3_3 = pd.merge(left = dart3_2, right = woman_chief, on = "종목코드", how = 'left')
new_dart3 = pd.merge(left = dart3_3, right = new_dart4, on = '종목코드', how = 'left')

new_dart3['여성임원비율'] = new_dart3['여성이사']/new_dart3['총이사']
new_dart3['ESG임원비율'] = new_dart3['ESG이사']/new_dart3['총이사']
r_dart3 = pd.concat([fgover[['종목코드','ESG임원비율']],new_dart3[['종목코드','ESG임원비율']]]).drop_duplicates("종목코드", keep = 'first')
new_dart3 = pd.merge(left = new_dart3.drop(['ESG임원비율'], axis = 1), right = r_dart3, on = '종목코드', how = 'left')

new_dart3.drop(['총이사','여성이사','ESG이사'], axis=1, inplace = True)

## 회계감사 (Full)
set_dart5 = dart5.drop_duplicates(['종목코드'], keep = 'first')
assess = list(set_dart5['감사의견'])
for i in range(0,set_dart5.shape[0]) :
    if '-' in assess[i] :
        assess[i] = '거절'
    elif '한정' in assess[i] :
        assess[i] = '한정'
    elif '거절' in assess[i] :
        assess[i] = '거절'
    else : assess[i] = '적정'
new_dart5 = pd.DataFrame({'종목코드':list(set_dart5['종목코드']),
                          '감사의견':assess})

## 지배구조보고서 준수율 : 결측치는 train set에서의 최솟값으로 대체

# 지배구조 핵심지표 준수율 & 금융회사 지배구조
new_dart6 = pd.concat([fgover[['종목코드','준수율']],dart6[['종목코드','준수율']]]).drop_duplicates("종목코드",keep ='first')
new_dart6 = pd.merge(left = sector_esg, right = new_dart6, how = 'left', on = '종목코드')

# train, test split
train_idx = raw_esg.iloc[:661,1]
train_min = new_dart6[new_dart6['종목코드'].isin(train_idx)].groupby('SICS_Sector_Kr')['준수율'].min().to_dict()
new_dart6['준수율'] = new_dart6['준수율'].fillna(pd.Series([train_min[x] for x in new_dart6['SICS_Sector_Kr']]))
new_dart6 = new_dart6[['종목코드','준수율']]

## 독립적 내부감사부서설치 & 금융회사 지배구조
dart6_2 = dart6[dart6['Q12'] == 1]
dart6_2['내부감사부서'] = 1
new_dart6_2 = pd.concat([fgover[['종목코드','내부감사부서']],dart6_2[['종목코드','내부감사부서']]]).drop_duplicates("종목코드",keep ='first')

## 내부감사기구에 회계전문가 or 재무전문가 존재 여부 & 금융회사 지배구조
dart6_3 = dart6[dart6['Q13'] == 1]
dart6_3['감사전문가'] = 1
new_dart6_3 = pd.concat([fgover[['종목코드','감사전문가']],dart6_3[['종목코드','감사전문가']]]).drop_duplicates("종목코드",keep ='first')

## 배당공지 여부
r_dart7 = dart7.drop_duplicates('종목코드',keep='first')
r_dart7['배당공지'] = 1
new_dart7 = r_dart7[['종목코드','배당공지']]

C:\Users\user\AppData\Local\Temp\ipykernel_13460\4262756820.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dart3 = dart3[dart3['등기임원여부'] != '미등기임원'][dart3['등기임원여부'] != '미등기']
C:\Users\user\AppData\Local\Temp\ipykernel_13460\4262756820.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dart6_1['이사회독립성'] = '독립'
C:\Users\user\AppData\Local\Temp\ipykernel_13460\4262756820.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ceo1 = dart3[dart3['직위'].str.contains('의장')][-dart3['직위'].str.contains('대표')][-dart3['담당업무'].str.contains('대표')]
C:\Users\user\AppData\Local\Temp\ipykernel_13460\4262756820.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ce

In [6]:
### KIND

## 공시우수법인 : 3년 간 공시우수법인 여부 (중복제거)
kind1 = kind1[(kind1['시상연도'] == 2022) | (kind1['시상연도'] == 2021) | (kind1['시상연도'] == 2020)]
kind1['공시우수법인'] = 1
new_kind1 = kind1.loc[:,['종목코드','공시우수법인']].drop_duplicates(['종목코드'],keep="first")

## 횡령3y : 3년 간 횡령 사실 여부 (중복제거)
new_kind2 = pd.DataFrame({'종목코드':np.unique(list(kind2['종목코드'])),
                          '횡령3y':np.ones(len(np.unique(list(kind2['종목코드']))))})

## 지배구조보고서 공시 & 금융회사 지배구조
r_kind3 = kind3[kind3['정정여부'] == 0]
r_kind3['지배구조공시'] = 1
r_kind3 = r_kind3.drop_duplicates("종목코드",keep='first')
new_kind3 = pd.concat([r_kind3[['종목코드','지배구조공시']],fgover[['종목코드','지배구조공시']]]).drop_duplicates("종목코드",keep ='first')

C:\Users\user\AppData\Local\Temp\ipykernel_13460\83082914.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_kind3['지배구조공시'] = 1


In [7]:
### KRX & KSA & 공정위위반 & 지배구조

## krx
krx['불성실공시'] = 1
krx = krx.drop_duplicates("종목코드", keep = 'first')
new_krx = krx[['종목코드','불성실공시']]

## 지속가능경영보고서 발간
ksa['지가경발간'] = 1
new_ksa = ksa[['종목코드','지가경발간']]

## 공정위위반
new_ill = ill[['종목코드','합계']].groupby(['종목코드']).sum()

## 지배구조 이사참석률 : 결측치는 train set에서의 최솟값으로 대체
new_gover = pd.merge(left = sector_esg[['종목코드','SICS_Sector_Kr']], right = gover.iloc[:,[3,5]], how = 'left', on = '종목코드')

# train, test split
train_idx = raw_esg.iloc[:661,1]
train_min = new_gover[new_gover['종목코드'].isin(train_idx)].groupby('SICS_Sector_Kr')['이사참석률'].min().to_dict()
new_gover['이사참석률'] = new_gover['이사참석률'].fillna(pd.Series([train_min[x] for x in new_gover['SICS_Sector_Kr']]))
new_gover = new_gover[['종목코드','이사참석률']]

C:\Users\user\AppData\Local\Temp\ipykernel_13460\130979234.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  krx['불성실공시'] = 1


### Data Merge

In [8]:
G_grade = raw_esg[['회사명','종목코드','SICS_Sector_Kr','G_GRADE']]
for i in ['1','2','3','5','6','7','6_2','6_3'] :
    G_grade = pd.merge(left = G_grade, right = globals()['new_dart{}'.format(i)], how = "left", on = "종목코드")

for i in [1,2,3] :
    G_grade = pd.merge(left = G_grade, right = globals()['new_kind{}'.format(i)], how = "left", on = "종목코드")
    
for i in ['new_krx','new_ksa','new_ill'] :
    G_grade = pd.merge(left = G_grade, right = globals()[i], how = "left", on = "종목코드")
    
G_grade = pd.merge(left = G_grade, right = new_gover[['종목코드','이사참석률']], how = 'left', on = '종목코드')
G_grade = G_grade.fillna(0)

### Governance Key Risk Index

In [9]:
### key risk 

## 지속가능경영보고서
gkr_k1 = G_grade[['종목코드','SICS_Sector_Kr']]
gkr_k1['지속가능경영보고서'] = (G_grade['지가경공시'] + G_grade['지가경발간'])/2

## 불성실 공시
gkr_k2 = pd.get_dummies(G_grade[['종목코드','불성실공시']], columns = ['불성실공시'])[['종목코드','불성실공시_0.0']].rename(columns = {'불성실공시_0.0':'불성실공시'})

## 횡령
gkr_k3 = G_grade[['종목코드']]
gkr_k3['횡령'] = np.abs(G_grade['횡령3y'] - 1)

## 이사회 독립성
gkr_k4 = pd.get_dummies(G_grade[['종목코드','이사회독립성']], columns = ['이사회독립성'])[['종목코드','이사회독립성_독립']].rename(columns = {'이사회독립성_독립':'이사회독립성'})

## 회계감사
gkr_k5 = pd.get_dummies(G_grade[['종목코드','감사의견']], columns = ['감사의견'])
gkr_k5['회계감사'] = gkr_k5['감사의견_적정'] + gkr_k5['감사의견_한정'] * 0.5
gkr_k5 = gkr_k5[['종목코드','회계감사']]

## 기타 (ESG임원비율,지배구조 준수율,주주배당,사외이사비율,공정위위반,내부감사부서설치,감사전문가,이사참석률)
gkr_etc1 = G_grade[['종목코드','ESG임원비율','준수율','배당공지','사외이사비율']]

gkr_etc2 = pd.get_dummies(G_grade[['종목코드','합계']], columns = ['합계'])
gkr_etc2['공정위위반'] = gkr_etc2['합계_0.0'] + gkr_etc2['합계_1.0'] * 0.5
gkr_etc2 = gkr_etc2[['종목코드','공정위위반']]

gkr_etc3 = G_grade[['종목코드','내부감사부서','감사전문가','이사참석률']]
gkr_etc3['이사참석률'] = gkr_etc3['이사참석률']/100

## data merge
gkr = pd.merge(left = gkr_k1, right = gkr_k2, how = 'left', on = '종목코드')
gkr = pd.merge(left = gkr, right = gkr_k3, how = 'left', on = '종목코드')
gkr = pd.merge(left = gkr, right = gkr_k4, how = 'left', on = '종목코드')
gkr = pd.merge(left = gkr, right = gkr_k5, how = 'left', on = '종목코드')
gkr = pd.merge(left = gkr, right = gkr_etc1, how = 'left', on = '종목코드')
gkr = pd.merge(left = gkr, right = gkr_etc2, how = 'left', on = '종목코드')
gkr = pd.merge(left = gkr, right = gkr_etc3, how = 'left', on = '종목코드')

C:\Users\user\AppData\Local\Temp\ipykernel_13460\1222327624.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gkr_k1['지속가능경영보고서'] = (G_grade['지가경공시'] + G_grade['지가경발간'])/2
C:\Users\user\AppData\Local\Temp\ipykernel_13460\1222327624.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gkr_k3['횡령'] = np.abs(G_grade['횡령3y'] - 1)
C:\Users\user\AppData\Local\Temp\ipykernel_13460\1222327624.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [10]:
### weight matrix
sics = pd.DataFrame({'소비재':[1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,0,0,0,0],
                     '추출물 및 광물 처리':[1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,0,0,0],
                     '금융':[1/15,1/15,1/15,1/15,1/15,1/15,1/15,1/15,1/15,0,2/15,2/15,2/15],
                     '식음료':[1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,0,0,0,0],
                     '헬스케어':[1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,0,0,0,0],
                     '인프라':[1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,0,1/12,1/12,1/12],
                     '재생가능 자원 및 대체 에너지':[1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,0,0,0],
                     '자원 변환':[1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,1/10,0,0,0],
                     '서비스':[1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,0,0,0,0],
                     '기술 및 통신':[1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,1/12,0,1/12,1/12,1/12],
                     '운송':[1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,0,0,0,0]})

In [11]:
### key risk : matrix 곱
a = [[0 for col in range(12)] for row in range(gkr.shape[0])]
for i in range(gkr.shape[0]) :
    a[i] = sics[gkr['SICS_Sector_Kr'][i]].tolist()

G_grade['G_risk'] = np.diag(np.array(gkr.iloc[:,2:]) @ np.array(a).T)
G_grade = G_grade.drop(['감사의견','이사참석률','합계'],axis=1)

### Final Governance Data

In [12]:
### modeling에 필요한 data split stratified 작업
G_group = [0 for col in range(G_grade.shape[0])]
for i in range(G_grade.shape[0]) :
    if G_grade['G_GRADE'].astype(int)[i] in [1,2,3] :
        G_group[i] = "G1"
    elif G_grade['G_GRADE'].astype(int)[i] in [4,5,6] :
        G_group[i] = "G2"
    elif G_grade['G_GRADE'].astype(int)[i] in [7,8,9] : 
        G_group[i] = "G3"
    else : G_group[i] = "NA"
        
G_grade['Stratify'] = G_grade['SICS_Sector_Kr'] + "_" + G_group

### 최종 데이터 추출
G_grade.to_excel("C:/Users/user/Desktop/ESG데이터/G_final.xlsx",index = False)
G_grade.to_excel("C:/Users/user/Desktop/ESG데이터/ESG/G_final.xlsx",index = False)
G_grade.to_excel("C:/Users/user/Desktop/ESG데이터/ESGI/G_final.xlsx",index = False)
G_grade.to_excel("C:/Users/user/Desktop/ESG데이터/Tableau/G_final.xlsx",index = False)